# Note

This code is for demonstration of Newton's in `Python`. Please contact Luca Steyn to report any errors.

# Imports

The code depends only on `numpy`.

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import warnings

## Functions for Binary Logistic Regression with Newton's method 

In [2]:
def sigmoid(z):
    """
    Calculate sigmoid function to map logits to probabilities.
    """
    return 1. / (1. + np.exp(-z))

def cross_entropy_loss(X, y, beta):
    """
    Compute the binary cross-entropy for logistic regression.
    """
    z = X @ beta
    p = sigmoid(z)
    loss = -np.sum(y*np.log(p) + (1-y)*np.log(1-p))
    return loss

def _gradient(X, y, beta):
    """
    Compute the gradient of the binary cross-entropy for logistic regression.
    """
    z = X @ beta
    p = sigmoid(z)
    return X.T @ (p - y)

def _hessian(X, beta):
    """
    Compute the Hessian matrix for logistic regression.
    """
    z = X @ beta
    p = sigmoid(z)
    diag_entry = p*(1.-p)
    diag_entry = diag_entry.ravel() # Note np.diag behaves differently for 1D (desired) and 2D arrays
    U = np.diag(diag_entry)
    return X.T @ U @ X

## Generate toy data

In [3]:
# Simulated data
np.random.seed(42)
n, p = 5, 2
X = np.random.randn(n, p)
Xext = np.hstack((np.ones((n, 1)), X))  # Add a column of ones for the intercept
true_beta = np.random.randn(p + 1).reshape(-1,1)
y = (sigmoid(Xext @ true_beta + 0.5 * np.random.randn(n).reshape(-1,1)) > 0.5).astype(int)

In [4]:
X.shape

(5, 2)

In [5]:
Xext.shape

(5, 3)

In [6]:
y.shape

(5, 1)

## Example of calculating loss, gradient and hessian

In [7]:
cross_entropy_loss(Xext, y, true_beta)

2.4781507115540538

In [8]:
_gradient(Xext, y, true_beta)

array([[0.86440716],
       [0.99750596],
       [0.39186338]])

In [9]:
_hessian(Xext, true_beta)

array([[1.14434118, 0.40025429, 0.56484959],
       [0.40025429, 0.7104337 , 0.40857727],
       [0.56484959, 0.40857727, 0.76347483]])

## Generate data for example

In [10]:
# Simulated data
np.random.seed(42)
n, p = 100, 3
X = np.random.randn(n, p)
Xext = np.hstack((np.ones((n, 1)), X))  # Add a column of ones for the intercept
true_beta = np.random.randn(p + 1).reshape(-1,1)
y = (sigmoid(Xext @ true_beta + 0.5 * np.random.randn(n).reshape(-1,1)) > 0.5).astype(int)

## Code for Newton's method

In [11]:
def newton_step(X, y, beta):
    """
    Compute the Newton step and decrement.
    """
    assert beta.shape[1] == 1, "beta must be a column vector - array with shape (?,1)."
    assert y.shape[1] == 1, "y must be a column vector - array with shape (?,1)."
    assert X.shape[1] == beta.shape[0], "X and beta must have matching dimensions."
    assert X.shape[0] == y.shape[0], "X and y must have same length."
    gradient = _gradient(X, y, beta)
    hessian = _hessian(X, beta)
    step = -1. * np.linalg.inv(hessian) @ gradient
    decrement = -1. * gradient.T @ step
    return step, gradient, decrement

In [12]:
newton_step(Xext, y, true_beta)

(array([[-0.67041886],
        [-0.44021255],
        [ 0.59345487],
        [ 0.77024027]]),
 array([[  8.23836731],
        [  4.60434928],
        [ -4.65907161],
        [-10.79326082]]),
 array([[18.62840211]]))

In [13]:
def backtracking_line_search(X, y, beta, step, decrement, initial_step_size=1.0, alpha1=0.8, alpha2=0.5):
    """
    Perform backtracking line search to find the step size using Newton's decrement.
    """
    t = initial_step_size
    loss_fn = cross_entropy_loss(X, y, beta)
    # Note we add the step since it is already the correct direction and use decrement on RHS
    while cross_entropy_loss(X, y, beta + t * step) > loss_fn - alpha2 * t * decrement:
        t *= alpha1
    return t

In [14]:
step, gradient, decrement = newton_step(Xext, y, true_beta)
backtracking_line_search(Xext, y, true_beta, step, decrement, initial_step_size=1.0, alpha1=0.8, alpha2=0.5)

1.0

In [15]:
def newton_method_logistic_regression(X, y, num_iterations=100, tol=1e-4, alpha1=0.9, alpha2=0.3, random_state=37):
    """
    Perform logistic regression using Newton's method with backtracking line search.
    """
    n, p = X.shape
    np.random.seed(random_state)
    beta = np.random.normal(size=X.shape[1]).reshape(-1,1)
    for i in range(num_iterations):
        step, gradient, decrement = newton_step(X, y, beta)
        
        # Perform backtracking line search to find the step size
        step_size = backtracking_line_search(X, y, beta, step, decrement, alpha1=alpha1, alpha2=alpha2)
        
        # Update the coefficients
        beta += step_size * step
        
        # Print log-likelihood every 10 iterations for monitoring
        if (i+1) % 10 == 0:
            print(f"Iteration {i + 1}: Loss {cross_entropy_loss(X, y, beta)}")
        
        # Check for convergence
        if np.linalg.norm(gradient) < tol:
            print(f"Converged after {i + 1} iterations")
            break
        
        if (i+1) == num_iterations:
            warnings.warn('Algorithm did not converge. Returning estimates of last update. Increase num_iterations or tol.')
    
    return beta

## Example on generated data

### Train

In [16]:
beta_est = newton_method_logistic_regression(Xext, y)

Iteration 10: Loss 9.828885730444613
Converged after 10 iterations


### Get training accuracy

Note we do not consider overfitting nor generalization of the model here. It is purely to demonstrate the workings of Newton's method. 

In [17]:
probs = sigmoid(Xext @ beta_est)
preds = [1 if p>0.5 else 0 for p in probs]

In [18]:
np.mean(preds == y.ravel())

0.94

In [19]:
print(classification_report(y.ravel(), preds))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96        77
           1       0.87      0.87      0.87        23

    accuracy                           0.94       100
   macro avg       0.92      0.92      0.92       100
weighted avg       0.94      0.94      0.94       100



#### E.g. where it doesn't converge because we do max 1 step

In [20]:
beta_est = newton_method_logistic_regression(Xext, y, num_iterations=1)

C:\Users\lucas\AppData\Local\Temp\ipykernel_10860\3840427108.py:27: UserWarning: Algorithm did not converge. Returning estimates of last update. Increase num_iterations or tol.
  warnings.warn('Algorithm did not converge. Returning estimates of last update. Increase num_iterations or tol.')


## Compare results with `sklearn`

Note that `sklearn` uses more sophisticated optimization algorithms than the standard Newton's method with backtracking. It is optimized to be stable and efficient. 

In [21]:
# sklearn
model = LogisticRegression(penalty=None)

In [22]:
model.fit(X, y.ravel())

LogisticRegression(penalty=None)

In [23]:
preds_sklearn = model.predict(X)

In [24]:
np.mean(preds_sklearn == y.ravel())

0.94

In [25]:
df_results = pd.DataFrame({
    'sklearn': np.concatenate((model.intercept_, model.coef_.ravel())),
    'custom': beta_est.ravel()
})
df_results['abs_diff'] = np.abs(df_results['sklearn'] - df_results['custom'])
df_results

,sklearn,custom,abs_diff
0,-6.857030,-1.071223,5.785808
1,-5.807819,-1.775366,4.032453
2,6.365563,0.240600,6.124963
3,6.616918,2.584335,4.032583


In [26]:
true_beta

array([[-0.82899501],
       [-0.56018104],
       [ 0.74729361],
       [ 0.61037027]])

## Extreme cases in the Loss

Note that the cross-entropy is not defined if $p$ is zero or one. As $p$ becomes close to zero or one, the cross-entropy has a term converging to $\log(0)$ which diverges to $-\infty$. One approach is to add a small constant in the $\log(\cdot)$ part of the cross-entropy loss. Furthermore, note that perfect separation of the classes introduces further challenges. (See [this link](https://stats.stackexchange.com/questions/451119/proving-mles-undefined-for-logistic-regression-with-separable-classes-in-p-dime) where the following was found)

### Perfect Separation in Logistic Regression

If the classes are linearly separable, there exists a hyperplane that can completely distinguish between the two classes (_i.e._, all positive examples are on one side of the hyperplane and all negative examples are on the other). Mathematically, this means there exists a vector $\boldsymbol{v}$ such that:

- $\boldsymbol{x}_i^{\top}\boldsymbol{v} > 0$ when $y_i = 1$, and
- $\boldsymbol{x}_i^{\top}\boldsymbol{v} < 0$ when $y_i = 0$.

This implies that we can find a margin $a > 0$ such that:

- $\boldsymbol{x}_i^{\top}\boldsymbol{v} \geq a$ when $y_i = 1$, and
- $\boldsymbol{x}_i^{\top}\boldsymbol{v} \leq -a$ when $y_i = 0$.

In this case, the logistic regression model can keep increasing the magnitude of $\boldsymbol{v}$ (scaling it by a factor $\xi$) to make the predicted probabilities $p_i = \sigma(\boldsymbol{x}_i^{\top} \boldsymbol{\beta})$ for the positive class closer to 1 and for the negative class closer to 0. As $\xi \to \infty$, the predicted probabilities $p_i$ for $y_i = 1$ approach 1, and $p_i$ for $y_i = 0$ approach 0.

The likelihood function for the logistic regression model, given the observations $\mathrm{X}$ and responses $\boldsymbol{y}$, is:

$$
\mathscr{L}(\boldsymbol{\beta}) = \prod_{i : y_i = 1} \sigma(\boldsymbol{x}_i^{\top} \boldsymbol{\beta}) \prod_{i : y_i = 0} (1 - \sigma(\boldsymbol{x}_i^{\top} \boldsymbol{\beta})),
$$

where $\sigma(x) = \frac{1}{1 + e^{-x}}$.

For perfect separation:

- As $\xi \to \infty$, $\boldsymbol{x}_i^{\top} \boldsymbol{\beta} \to \infty$ for $y_i = 1$ leading to $\sigma(x_i^{\top} \beta) \to 1$.
- Similarly, $\boldsymbol{x}_i^{\top} \boldsymbol{\beta} \to -\infty$ for $y_i = 0$ leading to $1 - \sigma(x_i^{\top} \beta) \to 1$.

Thus, the likelihood approaches 1 as $\xi \to \infty$. Therefore, the maximum likelihood estimate does not exist in the conventional sense; instead, the coefficients diverge to infinity.

Given the perfect separation, there is no unique MLE because:

- Any direction $\boldsymbol{v}$ that correctly separates the data can be scaled to arbitrarily large magnitudes, leading to infinite possible solutions where the likelihood approaches its maximum value of 1.
- Therefore, some coefficients $\boldsymbol{\beta}$ may diverge to infinity, while others may remain finite or become zero depending on their contribution to the separation.

#### Effect on Hessian

- When the model becomes extremely confident (_i.e._, $p_i$ is very close to 1 or 0), the term $p_i(1 - p_i)$ in the diagonal entries of $\mathrm{U}$ in the Hessian becomes very close to zero.
- This causes the Hessian matrix to approach singularity, making it numerically unstable to compute the inverse or use in Newton's method.

If $p_i \approx 1$ or $p_i \approx 0$, then $p_i(1 - p_i) \approx 0$, causing the Hessian to have near-zero diagonal elements.

There are a few ways to address these challenges, like:

- **Regularization:** Adding a small regularization term (like L2 regularization) to the loss function can help keep the Hessian from becoming singular by adding a positive constant to the diagonal.
- **Alternative Algorithms:** Use alternative optimization algorithms that do not rely on the Hessian inversion, like gradient descent or quasi-Newton methods (_e.g._, BFGS).

In [27]:
# Simulated data with higher dimension and perfect separation
np.random.seed(42)
n, p = 100, 10
X = np.random.randn(n, p)
Xext = np.hstack((np.ones((n, 1)), X))  # Add a column of ones for the intercept
true_beta = np.random.randn(p + 1).reshape(-1,1)
y = (sigmoid(Xext @ true_beta + 0.5 * np.random.randn(n).reshape(-1,1)) > 0.5).astype(int)

In [28]:
beta_est = newton_method_logistic_regression(Xext, y)

probs = sigmoid(Xext @ beta_est)
preds = [1 if p>0.5 else 0 for p in probs]

np.mean(preds == y.ravel())

Iteration 10: Loss nan
Converged after 16 iterations


C:\Users\lucas\AppData\Local\Temp\ipykernel_10860\4204095915.py:13: RuntimeWarning: divide by zero encountered in log
  loss = -np.sum(y*np.log(p) + (1-y)*np.log(1-p))
C:\Users\lucas\AppData\Local\Temp\ipykernel_10860\4204095915.py:13: RuntimeWarning: invalid value encountered in multiply
  loss = -np.sum(y*np.log(p) + (1-y)*np.log(1-p))


1.0

In [29]:
print(classification_report(y.ravel(), preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       1.00      1.00      1.00        70

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



### Adding small constant to the cross-entropy

Note this does not deal with the optimization problem. It only prevents the loss from becoming `nan`. 

In [30]:
def cross_entropy_loss(X, y, beta, eps=1e-08):
    """
    Compute the binary cross-entropy for logistic regression.
    """
    z = X @ beta
    p = sigmoid(z)
    loss = -np.sum(y*np.log(p + eps) + (1-y)*np.log(1-p + eps))
    return loss

def _gradient(X, y, beta):
    """
    Compute the gradient of the binary cross-entropy for logistic regression.
    """
    z = X @ beta
    p = sigmoid(z)
    return X.T @ (p - y)

def _hessian(X, beta):
    """
    Compute the Hessian matrix for logistic regression.
    """
    z = X @ beta
    p = sigmoid(z)
    diag_entry = p*(1.-p)
    diag_entry = diag_entry.ravel()
    U = np.diag(diag_entry)
    return X.T @ U @ X

In [31]:
beta_est = newton_method_logistic_regression(Xext, y)

probs = sigmoid(Xext @ beta_est)
preds = [1 if p>0.5 else 0 for p in probs]

np.mean(preds == y.ravel())

Iteration 10: Loss 0.03997938727492323
Converged after 16 iterations


1.0

Now we see the loss is defined because we avoid the risk of $\log(0)$. Note that $p$ and $1-p$ are always between zero and one. So $p+\epsilon$ and $(1-p)+\epsilon$ are always postive.

## Instability due to class separation

The separating hyperplane is not unique when the classes are easily linearly separable. For example, different initial values will give different solutions. One way to deal with this problem is through regularization.

In [32]:
seeds = [1, 11, 111, 1111]

for seed in seeds:
    beta_est = newton_method_logistic_regression(Xext, y, random_state=seed)
    probs = sigmoid(Xext @ beta_est)
    preds = [1 if p>0.5 else 0 for p in probs]
    acc = np.mean(preds == y.ravel())
    print(f'For seed {seed}, the estimates are:\n{np.round(beta_est.flatten(),3)}\nAnd the accuracy is: {acc:.2%}\n\n')

Iteration 10: Loss 0.09204935918079218
Converged after 16 iterations
For seed 1, the estimates are:
[ 58.359  20.091   4.395 -15.333  30.707  29.773  39.516  30.235  53.845
 -24.835  36.248]
And the accuracy is: 100.00%


Iteration 10: Loss 0.09716928003398165
Converged after 17 iterations
For seed 11, the estimates are:
[ 63.431  21.477   4.244 -16.607  34.297  32.744  42.36   32.94   58.509
 -27.534  38.528]
And the accuracy is: 100.00%


Iteration 10: Loss 0.12023484081627421
Converged after 17 iterations
For seed 111, the estimates are:
[ 62.463  21.569   4.949 -16.324  32.434  31.847  42.492  32.427  57.588
 -26.446  38.865]
And the accuracy is: 100.00%


Iteration 10: Loss 0.21046852198287502
Converged after 17 iterations
For seed 1111, the estimates are:
[ 59.269  20.399   4.442 -15.568  31.233  30.265  40.099  30.707  54.665
 -25.256  36.804]
And the accuracy is: 100.00%




### Add L2 penalty

Note we can add the L2 norm as a penalty to improve the instability. Remark that the code is set up to include an intercept, so we need to remove it from the vector and add a zero in the place of the intercept to avoid also regularizing it. Alternatively, the model can be fitted to the data without an intercept and the intercept can be estimated afterwards. 

In [33]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def get_regularization_term(beta, fit_intercept):
    if fit_intercept:
        reg_term = np.r_[[[0]], beta[1:,:]]  # Regularize only coefficients, not intercept
    else:
        reg_term = beta
    return reg_term

def cross_entropy_loss(X, y, beta, lambda_, fit_intercept=True, eps=1e-04):
    """
    Compute the binary cross-entropy for logistic regression with L2 regularization.
    """
    z = X @ beta
    p = sigmoid(z)
    reg_term = get_regularization_term(beta, fit_intercept)
    loss = -np.sum(y * np.log(p + eps) + (1. - y) * np.log(1. - p + eps)) + (lambda_ / 2.) * np.linalg.norm(reg_term)**2
    return loss

def _gradient(X, y, beta, lambda_, fit_intercept=True):
    """
    Compute the gradient of the binary cross-entropy for logistic regression with L2 regularization.
    """
    z = X @ beta
    p = sigmoid(z)
    reg_term = get_regularization_term(beta, fit_intercept)
    grad_ce = X.T @ (p - y)
    full_grad = grad_ce + lambda_ * reg_term
    return full_grad, grad_ce

def _hessian(X, beta, lambda_, fit_intercept=True):
    """
    Compute the Hessian matrix for logistic regression with L2 regularization.
    """
    z = X @ beta
    p = sigmoid(z)
    diag_entry = p * (1. - p)
    U = np.diag(diag_entry.ravel())
    reg_term = np.eye(beta.shape[0])
    if fit_intercept:
        reg_term[0, 0] = 0.  # Exclude the intercept (beta[0]) from regularization
    return X.T @ U @ X + lambda_ * reg_term

def newton_step(X, y, beta, lambda_, fit_intercept=True):
    """
    Compute the Newton step and decrement.
    """
    assert beta.shape[1] == 1, "beta must be a column vector - array with shape (?,1)."
    assert y.shape[1] == 1, "y must be a column vector - array with shape (?,1)."
    assert X.shape[1] == beta.shape[0], "X and beta must have matching dimensions."
    assert X.shape[0] == y.shape[0], "X and y must have the same length."
    
    gradient, grad_ce = _gradient(X, y, beta, lambda_, fit_intercept)
    hessian = _hessian(X, beta, lambda_, fit_intercept)
    
    # Use np.linalg.solve for stability instead of np.linalg.inv
    step = -np.linalg.solve(hessian, gradient)
    decrement = gradient.T @ np.linalg.solve(hessian, gradient)
    return step, gradient, decrement, grad_ce

def backtracking_line_search(X, y, beta, lambda_, fit_intercept, step, decrement, initial_step_size=1.0, alpha1=0.8, alpha2=0.5):
    """
    Perform backtracking line search to find the step size using Newton's decrement.
    """
    t = initial_step_size
    loss_fn = cross_entropy_loss(X, y, beta, lambda_, fit_intercept)
    
    while cross_entropy_loss(X, y, beta + t * step, lambda_, fit_intercept) > loss_fn - alpha2 * t * decrement:
        t *= alpha1
    return t

def newton_method_logistic_regression(X, y, lambda_, fit_intercept=True, num_iterations=500, tol=1e-4, alpha1=0.8, alpha2=0.3, random_state=1):
    """
    Perform logistic regression using Newton's method with backtracking line search.
    """
    n, p = X.shape
    np.random.seed(random_state)
    beta = np.random.normal(size=(X.shape[1], 1), scale=0.5)
    
    for i in range(num_iterations):
        step, gradient, decrement, grad_ce = newton_step(X, y, beta, lambda_, fit_intercept)
        
        # Perform backtracking line search to find the step size
        step_size = backtracking_line_search(X, y, beta, lambda_, fit_intercept, step, decrement, alpha1=alpha1, alpha2=alpha2)
        # Update the coefficients
        beta += step_size * step
        
        # Print loss every 10 iterations for monitoring
        if (i+1) % 10 == 0:
            print(f"Iteration {i + 1}: Loss {cross_entropy_loss(X, y, beta, lambda_, fit_intercept)}")
        
        # Check for convergence
        if np.linalg.norm(gradient) < tol or decrement < tol:
            print(f"Converged after {i + 1} iterations")
            break
        
        if (i+1) == num_iterations:
            warnings.warn('Algorithm did not converge. Returning estimates of last update. Increase num_iterations or tol.')
    
    return beta

In [34]:
# Scale data before applying regularization
from sklearn.preprocessing import StandardScaler

In [35]:
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
Xs = np.c_[np.ones(Xs.shape[0]), Xs]

In [36]:
beta_est = newton_method_logistic_regression(Xs, y, lambda_=0.1, fit_intercept=True, random_state=37, tol=1e-04)

Converged after 8 iterations


In [37]:
seeds = [1, 11, 111, 1111]

for seed in seeds:
    beta_est = newton_method_logistic_regression(Xs, y, lambda_=0.1, fit_intercept=True, random_state=seed, tol=1e-03)
    probs = sigmoid(Xext @ beta_est)
    preds = [1 if p>0.5 else 0 for p in probs]
    acc = np.mean(preds == y.ravel())
    print(f'For seed {seed}, the estimates are:\n{np.round(beta_est.flatten(),3)}\nAnd the accuracy is: {acc:.2%}\n\n')

Converged after 8 iterations
For seed 1, the estimates are:
[ 5.242  1.865  0.129 -1.374  2.6    2.456  3.531  2.136  3.583 -1.855
  3.686]
And the accuracy is: 100.00%


Converged after 8 iterations
For seed 11, the estimates are:
[ 5.242  1.865  0.129 -1.374  2.6    2.457  3.532  2.136  3.584 -1.855
  3.686]
And the accuracy is: 100.00%


Converged after 7 iterations
For seed 111, the estimates are:
[ 5.242  1.865  0.129 -1.374  2.6    2.456  3.531  2.136  3.584 -1.855
  3.686]
And the accuracy is: 100.00%


Converged after 8 iterations
For seed 1111, the estimates are:
[ 5.242  1.865  0.129 -1.374  2.6    2.456  3.532  2.136  3.584 -1.855
  3.686]
And the accuracy is: 100.00%




Note how stable the predictions are now. 

In [38]:
# Compare to sklearn
model = LogisticRegression(penalty='l2', C=1/0.1)

model.fit(Xs[:,1:], y.ravel())

preds_sklearn = model.predict(Xs[:,1:])

np.mean(preds_sklearn == y.ravel())

0.99

In [39]:
model.intercept_

array([5.2393375])

In [40]:
model.coef_

array([[ 1.86510055,  0.13062529, -1.37356496,  2.59694215,  2.45552131,
         3.53049662,  2.13600648,  3.58198536, -1.85296703,  3.68519357]])

In [41]:
df_results = pd.DataFrame({
    'sklearn': np.concatenate((model.intercept_, model.coef_.ravel())),
    'custom': beta_est.ravel()
})
df_results['abs_diff'] = np.abs(df_results['sklearn'] - df_results['custom'])
df_results['true_coef'] = true_beta
df_results

,sklearn,custom,abs_diff,true_coef
0,5.239337,5.241867,0.002529,1.399355
1,1.865101,1.865037,0.000063,0.924634
2,0.130625,0.129173,0.001452,0.059630
3,-1.373565,-1.373893,0.000328,-0.646937
4,2.596942,2.600014,0.003072,0.698223
5,2.455521,2.456486,0.000965,0.393485
6,3.530497,3.531518,0.001022,0.895193
7,2.136006,2.136466,0.000460,0.635172
8,3.581985,3.583583,0.001597,1.049553
9,-1.852967,-1.854871,0.001904,-0.535235


In [42]:
beta_est = newton_method_logistic_regression(Xs, y, lambda_=0.1, fit_intercept=True, random_state=35, 
                                             tol=1e-06, num_iterations=500, alpha1=0.8, alpha2=0.4)

Converged after 8 iterations


# Note:

The random initialization and hyperparameters can still influence the convergence. 

In [43]:
beta_est = newton_method_logistic_regression(Xs, y, lambda_=0.1, fit_intercept=True, random_state=35, 
                                             tol=1e-08, num_iterations=500, alpha1=0.8, alpha2=0.4)

Iteration 10: Loss 8.408888426542694
Iteration 20: Loss 8.408888426542653
Iteration 30: Loss 8.408888426542653
Iteration 40: Loss 8.408888426542653
Iteration 50: Loss 8.408888426542653
Iteration 60: Loss 8.408888426542653
Iteration 70: Loss 8.408888426542653
Iteration 80: Loss 8.408888426542653
Iteration 90: Loss 8.408888426542653
Iteration 100: Loss 8.408888426542653
Iteration 110: Loss 8.408888426542653
Iteration 120: Loss 8.408888426542653
Iteration 130: Loss 8.408888426542653
Iteration 140: Loss 8.408888426542653
Iteration 150: Loss 8.408888426542653
Iteration 160: Loss 8.408888426542653
Iteration 170: Loss 8.408888426542653
Iteration 180: Loss 8.408888426542653
Iteration 190: Loss 8.408888426542653
Iteration 200: Loss 8.408888426542653
Iteration 210: Loss 8.408888426542653
Iteration 220: Loss 8.408888426542653
Iteration 230: Loss 8.408888426542653
Iteration 240: Loss 8.408888426542653
Iteration 250: Loss 8.408888426542653
Iteration 260: Loss 8.408888426542653
Iteration 270: Loss 8

C:\Users\lucas\AppData\Local\Temp\ipykernel_10860\2234850326.py:99: UserWarning: Algorithm did not converge. Returning estimates of last update. Increase num_iterations or tol.
  warnings.warn('Algorithm did not converge. Returning estimates of last update. Increase num_iterations or tol.')


In [44]:
beta_est

array([[ 5.24161014],
       [ 1.8649807 ],
       [ 0.12914944],
       [-1.37387484],
       [ 2.59993072],
       [ 2.45637048],
       [ 3.53137494],
       [ 2.13631836],
       [ 3.5833902 ],
       [-1.85480143],
       [ 3.68573051]])